# Прогнозирование стоимости автомобиля BMW

# EDA обучающей выборки с AVITO

In [147]:
# Загрузка библиотек

import pandas as pd # импортируем необходимые библиотеки
import numpy as np
import re

# EDA тренировочной выборки ( с авито)

In [148]:
# Загрузка датасета и удаление пустых строк.

df = pd.read_csv('data_avito.csv')
df.dropna(subset = ['Марка'], inplace=True)  
df.dropna(subset = ['Price'], inplace=True)
df = df.reset_index(drop=True)      # индексирование по порядку
df.head()

,Марка,Модель,Поколение,Модификация,Год выпуска,Пробег,Состояние,Владельцев по ПТС,Тип кузова,Количество дверей,Тип двигателя,Коробка передач,Привод,Руль,Цвет,Комплектация,Price,Describe
0,BMW,5 серия,G30/G31 (2016—н. в.),530i 2.0 xDrive Steptronic (249 л.с.),2018.0,45357 км,не битый,1,седан,4.0,бензин,автомат,полный,левый,чёрный,NaN,2747000.0,"[<div class=""item-description""> <div class=""it..."
1,BMW,5 серия,F10/F11 (2009—2013),530d 3.0 xDrive AT (258 л.с.),2011.0,172467 км,не битый,2,седан,4.0,дизель,автомат,полный,левый,чёрный,NaN,1310000.0,"[<div class=""item-description""> <div class=""it..."
2,BMW,1 серия,F20/F21 (2011—2015),116i 1.6 AT (136 л.с.),2012.0,193000 км,не битый,4+,хетчбэк,5.0,бензин,автомат,задний,левый,белый,Базовая,520000.0,"[<div class=""item-description""> <div class=""it..."
3,BMW,X5,E70 рестайлинг (2010—2013),30d 3.0 AT (245 л.с.),2012.0,104153 км,не битый,1,внедорожник,5.0,дизель,автомат,полный,левый,серый,NaN,1759000.0,"[<div class=""item-description""> <div class=""it..."
4,BMW,X6,F16 (2014—н. в.),35i 3.0 AT (306 л.с.),2016.0,42100 км,не битый,2,внедорожник,5.0,бензин,автомат,полный,левый,белый,Базовая,2749900.0,"[<div class=""item-description""> <div class=""it..."


Проверим сначала общую информацию по датасету

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4015 entries, 0 to 4014
Data columns (total 18 columns):
Марка                4015 non-null object
Модель               4015 non-null object
Поколение            4015 non-null object
Модификация          4015 non-null object
Год выпуска          4015 non-null float64
Пробег               4015 non-null object
Состояние            4015 non-null object
Владельцев по ПТС    4015 non-null object
Тип кузова           4015 non-null object
Количество дверей    4015 non-null float64
Тип двигателя        4015 non-null object
Коробка передач      4015 non-null object
Привод               4015 non-null object
Руль                 4011 non-null object
Цвет                 4015 non-null object
Комплектация         3278 non-null object
Price                4015 non-null float64
Describe             4015 non-null object
dtypes: float64(3), object(15)
memory usage: 564.7+ KB


In [150]:
# Пустых значений не много, содержатся в столбцах Руль и Комплектация

df.isnull().sum()

Марка                  0
Модель                 0
Поколение              0
Модификация            0
Год выпуска            0
Пробег                 0
Состояние              0
Владельцев по ПТС      0
Тип кузова             0
Количество дверей      0
Тип двигателя          0
Коробка передач        0
Привод                 0
Руль                   4
Цвет                   0
Комплектация         737
Price                  0
Describe               0
dtype: int64

In [151]:
# Переименуем столбцы по аналогии с тестовым датасетом

df.columns = ([
    'brand',
    'model',
    'modelDate',
    'name',
    'productionDate',
    'mileage',
    'state',
    'owners',
    'bodyType',
    'numberOfDoors',
    'fuelType',
    'vehicleTransmission',
    'drive',
    'wheel',
    'color',
    'equipment',
    'price',
    'describe'
])

Рассмотрим последовательно каждый признак.

# Brand

In [152]:
df.brand.value_counts()

BMW    4015
Name: brand, dtype: int64

In [153]:
df = df.drop('brand', axis = 1)

# Model

In [154]:
df.model.value_counts()

5 серия                  841
X5                       640
3 серия                  618
X6                       409
7 серия                  387
X3                       278
1 серия                  184
X1                       156
X4                        65
6 серия                   65
M5                        64
X6 M                      43
5 серия GT                34
X5 M                      32
4 серия                   21
3 серия GT                19
8 серия                   19
X7                        17
M3                        16
M6                        15
4 серия Gran Coupe        14
2 серия                   14
6 серия GT                 9
M4                         9
i3                         8
Z4                         7
6 серия Gran Coupe         7
X2                         7
2 серия Grand Tourer       4
Z3                         3
i8                         3
321                        2
326                        2
X4 M                       1
M2            

Переименуем значения, приведем в соответствие тому значениям, к которым пришли в ноутбуке на kaggle.

In [155]:
dic_model = {
    '1 серия': '1ER',
    '2 серия': '2ER',
    '2 серия Active Tourer': '2ER',
    '2 серия Grand Tourer': '2ER',
    '3 серия': '3ER',
    '3 серия GT': '3ER',
    '321': '3ER',
    '326': '3ER',
    '4 серия': '4ER',
    '4 серия Gran Coupe': '4ER',
    '5 серия': '5ER',
    '5 серия GT': '5ER',
    '6 серия': '6ER',
    '6 серия GT': '6ER',
    '6 серия Gran Coupe': '6ER',
    '7 серия': '7ER',
    '8 серия': '8ER',
    'M2': 'delete',
    'M3': 'delete',
    'M4': 'delete',
    'M5': 'M5',
    'M6': 'delete',
    'X1': 'X1',
    'X2': 'X2',
    'X3': 'X3',
    'X4': 'X4',
    'X4 M': 'X4',
    'X5': 'X5',
    'X5 M': 'X5',
    'X6': 'X6',
    'X6 M': 'X6',
    'X7': 'X7',
    'Z3': 'delete',
    'Z4': 'Z4',
    'i3': 'delete',
    'i8': 'I8',
    }
df['model'] = df['model'].map(dic_model)

удалим лишние модели

In [156]:
df = df[df.model != 'delete']

# modelDate

В дате модели оставим только первую дату выпуска модели по аналогии с тестовой выборкой

In [157]:
df['modelDate'] = df['modelDate'].str.split('(').apply(lambda s: s[1].split('—')[0])

In [158]:
df.modelDate.value_counts()

2013    398
2007    329
2014    316
2009    285
2011    272
2008    255
2006    234
2010    227
2015    227
2012    214
2016    178
2005    134
2003    118
1998    100
2017     99
1995     95
2018     93
2002     78
2001     77
2000     42
1999     33
2004     31
1990     27
1988     26
1994     24
2019     21
1982      5
1989      5
1975      4
1977      4
1981      4
1976      3
1936      2
1937      2
1986      1
Name: modelDate, dtype: int64

In [159]:
df['modelDate'] = df['modelDate'].astype('int32')

In [160]:
df['modelDate'] = 2020 - df['modelDate']

In [161]:
df.rename(columns = {'modelDate':'modelAge'}, inplace = True) 

# Name

In [162]:
df.name[0]

'530i 2.0 xDrive Steptronic (249 л.с.)'

Выделим из данного столбца три: name, engineDisplacement и enginePower

In [163]:
def split_eng(s):
#   функция выделения из столбца name объема двигателя 
    pattern = re.compile('\d\W\d')
    if pattern.match(s) != None:
        s = s.split(' ')
        return s[0]
    elif len(s.split(' ')) > 2:
        s = s.split(' ')
        return s[1]
    else: return s 
    
def split_name(s):
#   функция выделения из столбца name модификации 
    pattern = re.compile('\d\W\d')
    if pattern.match(s) != None:
        return None
    else: 
        s = s.split(' ')
        return s[0] 

In [164]:
df['engineDisplacement'] = df['name'].apply(split_eng) 
df['enginePower'] = df['name'].str.split('(').apply(lambda s: float(s[1].split(' ')[0]))
df['name'] = df['name'].apply(split_name)

## engineDisplacement

In [165]:
sorted(df.engineDisplacement.unique())

['1.5',
 '1.6',
 '1.8',
 '1.9',
 '2.0',
 '2.2',
 '2.3',
 '2.5',
 '2.8',
 '2.9',
 '3.0',
 '3.0d',
 '3.0i',
 '3.0sd',
 '3.0si',
 '3.2',
 '3.4',
 '3.5',
 '3.6',
 '4.0',
 '4.4',
 '4.4i',
 '4.6is',
 '4.8',
 '4.8i',
 '4.8is',
 '4.9',
 '5.0',
 '5.4',
 '6.0',
 '6.6',
 'ED']

In [166]:
dic_engineDisplacement = {
    '1.5': 1.5,
    '1.6': 1.6,
    '1.8': 1.8,
    '1.9': 1.9,
    '2.0': 2.0,
    '2.2': 2.2,
    '2.3': 2.2,
    '2.5': 2.5,
    '2.8': 2.8,
    '2.9': 2.9,
    '3.0': 3.0,
    '3.0d': 3.0,
    '3.0i': 3.0,
    '3.0sd': 3.0,
    '3.0si': 3.0,
    '3.2': 3.2,
    '3.4': 3.4,
    '3.5': 3.5,
    '3.6': 3.6,
    '4.0': 4.0,
    '4.4': 4.4,
    '4.4i': 4.4,
    '4.6is': 4.6,
    '4.8': 4.8,
    '4.8i': 4.8,
    '4.8is': 4.8,
    '4.9': 4.9,
    '5.0': 5.0,
    '5.4': 5.4,
    '6.0': 6.0,
    '6.6': 6.6,
    'ED': 0,
}
df['engineDisplacement'] = df['engineDisplacement'].map(dic_engineDisplacement)

In [167]:
df = df[df['engineDisplacement'] != 0]

## enginePower

In [168]:
df.enginePower.value_counts()

184.0    480
249.0    344
306.0    324
218.0    294
190.0    271
        ... 
347.0      1
544.0      1
510.0      1
269.0      1
188.0      1
Name: enginePower, Length: 68, dtype: int64

In [169]:
df['enginePower'] = df['enginePower'].astype('int32')

## Name

In [170]:
df['name'] = df['name'].astype('str')

In [171]:
df.name.value_counts()

None     342
30d      336
35i      230
320i     204
20d      182
        ... 
535d       1
340i       1
525xd      1
520        1
335xi      1
Name: name, Length: 105, dtype: int64

# ProductionDate

In [172]:
df['productionDate']=(2020-df['productionDate'])

In [173]:
df['productionDate'].value_counts()

7.0     367
8.0     323
9.0     305
12.0    279
3.0     270
1.0     252
6.0     249
4.0     241
5.0     217
2.0     211
10.0    206
13.0    174
11.0    158
16.0     95
15.0     86
22.0     84
14.0     82
17.0     66
18.0     62
21.0     46
19.0     44
23.0     29
20.0     22
24.0     15
27.0     11
26.0      9
28.0      9
30.0      9
29.0      6
31.0      6
25.0      5
42.0      4
32.0      4
37.0      4
33.0      3
82.0      2
72.0      2
39.0      2
40.0      1
34.0      1
36.0      1
Name: productionDate, dtype: int64

In [174]:
df['productionDate'] = df['productionDate'].astype('int32')

In [175]:
df.rename(columns = {'productionDate':'age'}, inplace = True) 

# Mileage

In [176]:
df.mileage

0        45357 км
1       172467 км
2       193000 км
3       104153 км
4        42100 км
          ...    
4010     35000 км
4011      3844 км
4012     78000 км
4013    105600 км
4014       120 км
Name: mileage, Length: 3962, dtype: object

In [177]:
df['mileage'] = df['mileage'].str.split('\xa0').apply(lambda s: int(s[0]))   # уберем км из пробега

In [178]:
df['mileage'] = df['mileage'].astype('int32')

# Owners

In [179]:
# ПРиведем значения стобца  owners в соответствии со значениями на тестовой выборке

dic_owners = {'1': 1, '2': 2, '3': 3, '4+': 3}
df.owners = df.owners.map(dic_owners)

# bodyType

In [180]:
df.bodyType.value_counts()

седан          1747
внедорожник    1648
хетчбэк         262
купе            212
кабриолет        46
универсал        42
минивэн           5
Name: bodyType, dtype: int64

In [181]:
dic_bodyType = {
    'седан': 'sedan',
    'внедорожник': 'SUV',
    'хетчбэк': 'hatchback',
    'купе': 'coupe',
    'универсал': 'wagon',
    'кабриолет': 'cabriolet',
    'минивэн': 'van'
}
df['bodyType'] = df['bodyType'].map(dic_bodyType)

# numberOfDoors

In [182]:
df.numberOfDoors.value_counts()

5.0    1924
4.0    1741
2.0     264
3.0      33
Name: numberOfDoors, dtype: int64

In [183]:
df['numberOfDoors'] = df['numberOfDoors'].astype('int32')
df['numberOfDoors'] = df['numberOfDoors'].astype('str')

# fuelType

In [184]:
df.fuelType.value_counts()

бензин     2577
дизель     1380
гибрид        3
электро       2
Name: fuelType, dtype: int64

In [185]:
dic_fuelType = {
    'дизель': 'diesel',
    'бензин': 'petrol',
    'гибрид': 'hybrid',
    'электро': 'electro'
}
df['fuelType'] = df['fuelType'].map(dic_fuelType)

# vehicleTransmission

In [186]:
df.vehicleTransmission.value_counts()

автомат     3725
механика     195
робот         42
Name: vehicleTransmission, dtype: int64

In [187]:
dic_vehicleTransmission = {
    'автомат': 'AT',
    'механика': 'MT',
    'робот': 'AM'
}
df['vehicleTransmission'] = df['vehicleTransmission'].map(dic_vehicleTransmission)

# drive

In [188]:
df.drive.value_counts()

полный      2363
задний      1579
передний      20
Name: drive, dtype: int64

In [189]:
dic_drive = {
    'полный': '4wd',
    'задний': 'rwd',
    'передний': 'fwd'
}
df['drive'] = df['drive'].map(dic_drive)

# wheel

In [190]:
df.wheel.value_counts()

левый     3951
правый       7
Name: wheel, dtype: int64

In [191]:
df['wheel'] = df['wheel'].fillna('левый')

In [192]:
df = df[df.wheel=='левый']

In [193]:
df = df.drop('wheel', axis = 1)

# color

In [194]:
df.color.value_counts()

чёрный        1659
белый          605
серый          516
синий          495
серебряный     181
коричневый     145
красный        139
бежевый         77
зелёный         64
голубой         48
пурпурный        9
фиолетовый       7
оранжевый        5
золотой          4
жёлтый           1
Name: color, dtype: int64

In [195]:
dic_color = {
    'чёрный': 'black',
    'белый': 'white',
    'серый': 'grey',
    'синий': 'dark_blue',
    'серебряный': 'silver', 
    'красный': 'red',
    'коричневый': 'brown',
    'зелёный': 'green',
    'бежевый': 'biege',
    'голубой': 'blue',
    'пурпурный': 'purple',
    'оранжевый': 'orange',
    'зоротой': 'golden',
    'фиолетовый': 'violet',
    'жёлтый': 'yellow', 
}
df['color'] = df['color'].map(dic_color)

# equipment

In [196]:
df.equipment.value_counts()

Базовая                      1986
Base                          191
M Sport                       186
Luxury                        161
Business                      145
SE                            104
Exclusive                      48
M                              41
Sport Line                     33
Lifestyle                      32
Urban                          30
Pure Excellence                25
M Sport Локальная сборка       23
Luxury Line                    23
M Sport Experience             22
Sport                          22
Prestige                       19
Luxury Локальная сборка        14
Advantage                      11
M Sport Pure                   10
xLine                          10
M Sport Pro                     9
Xline                           9
SE Локальная сборка             9
Touring                         7
Executive                       7
Edition Exclusive               6
Business Локальная сборка       6
Pure Experience                 4
Business SKD  

# price

In [197]:
df.price.value_counts()

1400000.0    68
1100000.0    59
800000.0     59
950000.0     57
550000.0     54
             ..
1645000.0     1
915000.0      1
4085000.0     1
2463000.0     1
3916000.0     1
Name: price, Length: 829, dtype: int64

In [198]:
df['price'] = df['price'].astype('int32')

# describe

In [199]:
df['describe'][5]

'[<div class="item-description"> <div class="item-description-text" itemprop="description">\n<p>BMW X5, e53<br/>2004 года, мотор бешенный 4.8is (360сил), комплектация жирная: память, шторки, стеклопакеты!!открывания ворот, панорама, чёрный потолок, ксенон, спорт сиденья, регулировка заднего ряда и тд. <br/>Машина в нормальном состояние, много что сделано, птс оригинал. <br/>Без запретов и ограничений!<br/>Пробег реальный (на этих щитках при скрутке перестают работать температурные стрелки)<br/>есть окрасы, без криминала <br/>20 тапки на низком профиле, 315 ширина (диски и резина новые! Меньше месяца!)<br/>Под карбоновой пленкой дерево!</p><p>Пройдёмся по плюсам:<br/>Привода передние только поставил. <br/>Кпп было в ремонте, сейчас чувствует себя прекрасно.  (6 передач) <br/>Вместе с приводами поставил новые свечи, поменял масло двигатель, раздатка, мосты , антифриз, фильтра <br/>Свежий прямоточный выхлоп без катализаторов, 5.5 курит в сторонке (видео по запросу вышлю). <br/>Так же свеж

In [200]:
# Создадим и заполним поле ПТС на основании данных в описании к объявлению 
    
st_ptc = ['оригинальный птс' , 'оригинал птс', 'дубликат']

def code_ptc (ptc):
# "вытаскивание" информации о ПТС из описания

    d = ptc.lower()
    if (st_ptc[0] in d) or (st_ptc[1] in d):
        return 1
    elif st_ptc[2] in d:
        return 0
    else:
        return 1

df['technicalPassport'] = df['describe'].apply(code_ptc)
df = df.drop('describe', axis = 1)
df['technicalPassport'].value_counts()

1    3942
0      13
Name: technicalPassport, dtype: int64

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3955 entries, 0 to 4014
Data columns (total 18 columns):
model                  3955 non-null object
modelAge               3955 non-null int32
name                   3955 non-null object
age                    3955 non-null int32
mileage                3955 non-null int32
state                  3955 non-null object
owners                 3955 non-null int64
bodyType               3955 non-null object
numberOfDoors          3955 non-null object
fuelType               3955 non-null object
vehicleTransmission    3955 non-null object
drive                  3955 non-null object
color                  3951 non-null object
equipment              3224 non-null object
price                  3955 non-null int32
engineDisplacement     3955 non-null float64
enginePower            3955 non-null int32
technicalPassport      3955 non-null int64
dtypes: float64(1), int32(5), int64(2), object(10)
memory usage: 669.8+ KB


In [202]:
# Сохраним в файл.

df.to_csv('avito.csv', index=False)